<a href="https://colab.research.google.com/github/mfilipak/AFRAC_IA/blob/main/004_Visualiza%C3%A7%C3%B5es_usando_modelo_pr%C3%A9_treinado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# EM CONSTRUÇÃO

# Experimento 004
##Objetivo: Experimentos de visualização de word embeddings carregando modelo(s) treinado(s) no Experimento 002.
###Descrição: Experimentos iniciais para criação e visualização de word embeddings<br>
Dica: No COLAB Use CTRL SPACE ao invés de TAB para "autocompletar". Ex:pd.re [CTRL SPACE] vai mostrar uma lista contendo as funções e atributos que começam com pd.re (como read_csv, ...) 

In [110]:
from glob import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
import pprint

from gensim.models import Word2Vec
#from sentence_transformers import SentenceTransformer
import joblib
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

import seaborn as sns
import pandas as pd
import numpy as np
from mpl_toolkits import mplot3d
import plotly.express as px

In [2]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
#Copia os dados das NFEs do portal da cidadância pro drive virtual.
import requests  
file_url = "https://raw.githubusercontent.com//mfilipak/AFRAC_IA/main/MODELS/w2vec_pdt_5000"
r = requests.get(file_url, stream = True) 

with open("w2vec_pdt_5000", "wb") as file:  
    for block in r.iter_content(chunk_size = 1024): 
         if block:  
             file.write(block)


In [4]:
# Caso o treino tenha sido feito chamando a classe callback ela precisa ser declarada para permitir a carga

#TO DO: Não foi possível monitorar a loss durante o treino.
from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import Word2Vec
from gensim.models.callbacks import CallbackAny2Vec

#import logging
#logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

class callback(CallbackAny2Vec):
    '''Callback to print loss after each epoch.'''

    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        try:
          loss = model.running_training_loss
          print('Loss after epoch {}: {}'.format(self.epoch, loss))
        except:
          print(f"EXCEPTION - Epoch:{self.epoch}")
        self.epoch += 1



In [5]:
model = Word2Vec.load("w2vec_pdt_5000",  )

In [6]:
list(model.wv.vocab.keys())[:10]

['maza',
 'diluente',
 'p/',
 'epoxi',
 'dp-003',
 '5',
 'l',
 'coletor',
 'mat.perfuro',
 'cortantes']

In [82]:
test_doc = word_tokenize("banana prata".lower())
test_doc_vector = model.infer_vector(test_doc)
test = model.docvecs.most_similar(positive = [test_doc_vector])
test

[(2197, 0.8462918400764465),
 (4738, 0.8331982493400574),
 (1069, 0.8303892016410828),
 (840, 0.8127132654190063),
 (304, 0.8011035919189453),
 (2209, 0.7637394666671753),
 (833, 0.7607715725898743),
 (2478, 0.7427284717559814),
 (4366, 0.7394019365310669),
 (627, 0.7328137159347534)]

In [70]:
for i, _ in test:
  target =  model.docvecs[i]
  print(i, target.dot(test_doc_vector)/np.linalg.norm(target)/np.linalg.norm(test_doc_vector))

4383 0.6494874
4650 0.5587584
4088 0.54741865
407 0.5371437
1184 0.53489846
1461 0.5167127
1288 0.51626986
4725 0.51539415
903 0.5106894
1934 0.5100722


In [87]:
#Esse trecho de código copia o text_corpus do jeito que foi feito o treino.
#Posteriormente, vamos fazer com que isso não seja relevante montando all_vectors a partir da inferência
import requests  
file_url = "https://raw.githubusercontent.com//mfilipak/AFRAC_IA/main/DATASET/202201_NFe_NotaFiscalItem.zip"
r = requests.get(file_url, stream = True) 

with open("portal.zip", "wb") as file:  
    for block in r.iter_content(chunk_size = 1024): 
         if block:  
             file.write(block)

DATA_FILE = "portal.zip"
df = pd.read_csv(DATA_FILE, encoding="CP1252",sep=";")
print("O dataframe completo contém:",len(df),"linhas")
df3 = df[['DATA EMISSÃO','DESCRIÇÃO DO PRODUTO/SERVIÇO', 'CÓDIGO NCM/SH', 'CFOP']]
df3.columns = ["DATA", "DESCR", "NCM", "CFOP"]
df3 = df3[df3["NCM"]!=-1] #Filtrando NCMs = -1

text_lengths = np.array([len(_) for _ in df3['DESCR']])
df3 = df3[text_lengths>=3]

#Caso queira eliminar as repetições rodar a linha abaixo
df3 = df3.drop_duplicates(subset=["DESCR"])
random.seed(42)
text_corpus = random.sample(list(df3["DESCR"]),5000)
len(text_corpus)

O dataframe completo contém: 324056 linhas


5000

In [90]:
all_vectors = model.docvecs.vectors_docs

test_doc = word_tokenize("banana prata".lower())
test_doc_vector = model.infer_vector(test_doc)

cos_distances = all_vectors.dot(test_doc_vector)/np.linalg.norm(all_vectors, axis=-1)/np.linalg.norm(test_doc_vector)
for i in np.argsort(cos_distances)[::-1][:10]:
  print(text_corpus[i])


Banana comprida
FRUTA IN NATURA BANANA PRATA
BANANA NANICA/CATURRA CLIM PRIM
BANANA ESPECIE COMPRIDA
BANANA NANICA
OUTROS/BANANA PRATA
FRUTA IN NATURA, TIPO MELAO, ESPECIE AMARELO.
FARINHA DE ROSCA - PULLMAN
ALHO NATURAL CABEÇA BRANCA
DOCE DE BANANA COM 50UND


In [91]:
random.seed(0)
new_text_corpus = random.sample(list(df3["DESCR"]),5000)
len(new_text_corpus)

5000

In [93]:
all_vectors = model.docvecs.vectors_docs

test_doc = word_tokenize("banana prata".lower())
test_doc_vector = model.infer_vector(test_doc)

cos_distances = all_vectors.dot(test_doc_vector)/np.linalg.norm(all_vectors, axis=-1)/np.linalg.norm(test_doc_vector)
for i in np.argsort(cos_distances)[::-1][:10]:
  print(new_text_corpus[i])

IOGURTE NINHO MACA E BANANA 170 G
PLACA LCP RADIO D LAT 2,7MM 6F L: 160032879-025 ANVISA N : 80082910118
ABRAÇADEIRA, MATERIAL PLÁSTICO, COMPRIMENTO TOTAL 110 MM, LARGURA 2,50 MM,APLICAÇÃO
FORMA TUBO PUDIM/BOLO
RABADA
FECH PADO TRIO QUADRA ROS 701-80 MP16500 CR
FONTE 24VCC BIVOLT SAIDA DUPLA 3A 72W DIN CERTIFICADA
OUTROS OLEOS LUBRIFICANTES AUTOMOTIVOS                                     - Item 5
INTRINSECA | SPRINT: O METODO USADO NO GOOGLE PARA TESTAR E APLICAR NOVAS IDEIAS EM APENAS CINCO DIAS | Ed: 12017
CIL MESTRE FREIO REMANOFATURADO


In [94]:
stoplist = set('- DE E PARA X COM EM TIPO de A | DO C/ DA / P/ Ed O'.split(' '))
# Lowercase each document, split it by white space and filter out stopwords
texts = [[word for word in document.lower().split() if word not in stoplist]
         for document in new_text_corpus]

# Count word frequencies
from collections import defaultdict
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1

# Only keep words that appear more than once
#Ignorei esse filtro pois com poucos exemplos iria cortar quase tudo
processed_corpus = [[token for token in text if frequency[token] > 0] for text in texts]
pprint.pprint(processed_corpus[:5])

[['vavula', 'escape', 'tr4', '2010/...'],
 ['foice', 'rocadeira', 'tipo', 'l,', 'c/', 'cabo,', '1,2m'],
 ['sinapi', '4721', 'brita', '1', '(posto', 'jazida)'],
 ['componente',
  'cortador',
  'legumes,',
  'tipo',
  'manual,',
  'tipo',
  'acessório',
  'lâmina,',
  'material',
  'aço',
  'inoxidável'],
 ['fresa', 'canulada', '06,0', 'mm.', 'rms.80804050147']]


In [101]:
all_vectors = []
for pc in processed_corpus[:]:
  all_vectors += [model.infer_vector(pc)]
all_vectors = np.array(all_vectors)
all_vectors.shape

In [122]:
from sklearn.manifold import TSNE
tsne = TSNE(n_components=3, random_state=0)
vectors = tsne.fit_transform(all_vectors)

/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:783: FutureWarning:

The default initialization in TSNE will change from 'random' to 'pca' in 1.2.

/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:793: FutureWarning:

The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.



In [109]:
#all_vectors = model.docvecs.vectors_docs

test_doc = word_tokenize("erva mate".lower())
test_doc_vector = model.infer_vector(test_doc)

cos_distances = all_vectors.dot(test_doc_vector)/np.linalg.norm(all_vectors, axis=-1)/np.linalg.norm(test_doc_vector)
for i in np.argsort(cos_distances)[::-1][:10]:
  print(new_text_corpus[i])

CHA ERVA CIDREIRA
CHA FRUTAS CITRICAS CHILENO ECONOMICO CAIXA 10G/10 SACHES
ERVA MATE 1KG
FOSVITA 250G
2I BIOLOGICAL TEST - VAPOR - 3H - CX. C/ 50 UN.
milho, tipo: grão, aplicação: mungunzá (canjica) 500G
COLHER CHA BASIC - GOURMET MIX GX4070
TUBO DE CENTRIFUGA 15ML CX 500 UN
CAFE BICO PREMIUM SUPERIOR 500G LIC
MOSTARDA 250G


In [113]:
names = [""]*len(new_text_corpus)
for i in np.argsort(cos_distances)[::-1][:10]:
  names[i] = new_text_corpus[i]

In [118]:
table = [ [n,*c] for n, c in zip(names,list(np.random.random((len(names),3))*2-1)) ]
df = pd.DataFrame(table, columns=["Name", "X", "Y", "Z"])

In [ ]:
fig = px.scatter_3d(df, x="X", y="Y", z="Z", opacity=0.5, text=names)
fig.show()

In [127]:
table = [ [n,*c] for n, c in zip(new_text_corpus,list(vectors)) ]
df = pd.DataFrame(table, columns=["Name", "X", "Y", "Z"])
df

,Name,X,Y,Z
0,VAVULA DE ESCAPE TR4 2010/...,13.291286,2.806996,-3.719353
1,"FOICE ROcADEIRA TIPO L, C/ CABO, 1,2M",25.835499,8.114185,8.409164
2,SINAPI 4721 - BRITA 1 (POSTO JAZIDA),-24.365019,-11.026721,34.619678
3,"COMPONENTE CORTADOR LEGUMES, TIPO MANUAL, TIPO...",-2.412488,-10.134967,45.735321
4,"FRESA CANULADA DE 06,0 MM. RMS.80804050147",-14.490551,-33.751827,25.844044
...,...,...,...,...
4995,PINO GUIA PINCA FREIO L200 TRITON/ TR4,0.891163,-41.062313,13.278102
4996,FOICE ROCADEIRA COM CABO,-26.723814,6.837378,0.996389
4997,HASTE DA CHAVE HEXAGONAL 2.5mm,-8.203831,-13.179846,33.725689
4998,Papel sulfite A4 75g 210mmx297mm PT 500 FL,13.888993,12.229925,-19.150497


In [141]:
test_doc = word_tokenize("banana caturra".lower())
test_doc_vector = model.infer_vector(test_doc)

cos_distances = all_vectors.dot(test_doc_vector)/np.linalg.norm(all_vectors, axis=-1)/np.linalg.norm(test_doc_vector)
names = [""]*len(new_text_corpus)
for i in np.argsort(cos_distances)[::-1][:10]:
  print(cos_distances[i], new_text_corpus[i])
  names[i] = new_text_corpus[i]


0.9895864 COOKIES DE BANANA
0.7620131 OUTROS/BANANA PRATA
0.71935713 COCADA
0.70019054 1.36.40.03.19.046 - PARAFUSO CORTICAL O 4.0 X 46.0 MM COM SEXTAVADO  EM TITANIO
0.7000451 IMPRESSORA LX - CDUM174948
0.6917726 PARAF ACET BAIXO P TIT 6.5X15MM REF:04.43.19.65015
0.6903943 LUNHACA KG
0.69013315 ESPONJA DUPLA FACE COMUM
0.6898829 CALCO FEMORAL DISTAL MB V T5X05MM
0.6854244 PORTMARLEX TROCARTE 12X100MM, 01 TROCARTE + 01 OBTURADOR S LAMINA


In [ ]:
fig = px.scatter_3d(df, x="X", y="Y", z="Z", opacity=0.01, text=names)
fig.show()